# Install and Import 

In [7]:
# !pip install tensorflow==2.15.0
# !pip install gym==0.29.1
# !pip install keras
# !pip install keras-rl2
# %pip install scikit-learn

In [8]:
# %pip install ipykernel
# %pip install --upgrade nbformat
# %pip install stable-baselines3[extra]
# %pip install gymnasium==0.29.1

In [9]:
# %pip install stable-baselines3 plotly numpy pandas
# %pip install ipywidgets
# %pip install pandas_ta

In [10]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [11]:
import numpy as np
import pandas as pd
import random
import logging
import math
from model_config import Path
import os
import torch
import fnmatch

In [12]:
import gymnasium as gym
from gymnasium import Env
from gymnasium import spaces
from gymnasium.utils import seeding

In [13]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import SubprocVecEnv

In [14]:
import decimal
decimal.getcontext().prec = 28  # Increase precision

In [15]:
import plotly.express as px
import plotly.graph_objects as go

In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Используем GPU
    print("CUDA доступна. Работаем на GPU.")
else:
    device = torch.device("cpu")   # Используем CPU
    print("CUDA не доступна. Работаем на CPU.")


CUDA не доступна. Работаем на CPU.


# Open Data

## Description of changes

Основано на Model 2, Отличия:
- Удалил Штраф за попытки купить токены при отсутствии денег на них
$$
self.reward -= 5
$$

- Удалил Штраф за попытки продать токены при отсутствии самих токенов
$$
self.reward -= 5
$$

- Удалил штраф за Hold 
$$
self.reward -= 1
$$


## Settings

In [17]:
window_length = 48
nb_steps = 80000

model_num = 6
data_num = 1

seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed= seed)

## Open Data

In [18]:
def find_dataset_file(dnum, folder="data"):
    pattern = f"dataset_{dnum}D_*.csv"
    for filename in os.listdir(folder):
        if fnmatch.fnmatch(filename, pattern):
            return os.path.join(folder, filename)
    return None

In [19]:
data_path = find_dataset_file(data_num, folder= Path["processed"])
print(data_path)
df = pd.read_csv(data_path)
df.shape

/Users/mchomak/Projects/RL_for_Trading/data/processed/dataset_1D_Standart_26cols_26tkn_1t.csv


(103954, 26)

In [20]:
zero_close_prices = df[df['Close_orig'] == 0]
print(f"Number of zero 'Close' prices after scaling: {len(zero_close_prices)}")

Number of zero 'Close' prices after scaling: 0


In [21]:
unique_values = df['Asset_ID_encoded'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25])

In [22]:
value_counts = df['Asset_ID_encoded'].value_counts()
value_counts

Asset_ID_encoded
1     4388
23    4388
18    4388
0     4387
5     4387
16    4384
15    4384
6     4379
25    4372
19    4371
4     4367
14    4342
13    4334
20    4314
3     4313
8     4309
22    4219
9     4207
21    4093
24    3602
11    3589
10    3582
17    3120
7     2957
2     2931
12    1847
Name: count, dtype: int64

In [23]:
def split_by_asset_ids(df: pd.DataFrame, test_asset_ids: list):
    test_df = df[df['Asset_ID_encoded'].isin(test_asset_ids)]
    train_df = df[~df['Asset_ID_encoded'].isin(test_asset_ids)]
    return train_df, test_df

In [24]:
test_asset_ids = [21, 22, 23, 24, 25]
train_df, test_df = split_by_asset_ids(df = df, test_asset_ids = test_asset_ids)
print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

Training data shape: (83280, 26)
Testing data shape: (20674, 26)


# Game Rule

In [25]:
class TradingEnv(gym.Env):
    """
    Environment for training an agent to trade on the exchange using a continuous action space.
    """
    metadata = {'render.modes': ['human']}

    def __init__(self, df, mode):
        super(TradingEnv, self).__init__()

        # Save data and initialize parameters
        self.df = df.reset_index(drop=True)
        self.total_steps = len(self.df) - 1
        self.window_length = window_length
        self.mode = mode # test or train

        # Find indices where a new asset starts
        self.asset_start_indices = self._find_asset_start_indices()
        print(self.asset_start_indices)

        # Define action space: Continuous action between -1 and 1
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)

        # Define observation space
        num_features = len(self.df.columns) - 1 # Вычли Close_orig
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.window_length, num_features), dtype=np.float32)

        # Initialize trading parameters
        self.fee_cost = 0.001
        self.initial_balance = 1000  # Starting balance
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.fut_net_worth = self.initial_balance

        self.reward = 0
        self.current_step = self.window_length
        self.current_price = 0
        self.tokens_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        self.now_token = (self.df.loc[self.current_step, 'Asset_ID_encoded'] - 1)
        print(self.now_token)
        self.prev_token = self.now_token

        self.hist = {
            "current_step": [],
            'balance': [],
            'net_worth': [],
            'tokens_held': [],
            "token": [],
            "current_price": [],
            "reward": [],
            "action": [],
            'total_shares_sold': [],
            'total_sales_value': [],
        }

        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        log_file = Path["train_log"](model_num, data_num + 1)
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)

        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        self.logger.addHandler(file_handler)
        logging.getLogger().handlers = []


    def _find_asset_start_indices(self):
        """
        Find the indices in the DataFrame where a new asset starts.
        """
        asset_ids = self.df['Asset_ID_encoded']
        start_indices = {asset_ids[0]:0}
        for i in range(1, len(asset_ids)):
            if asset_ids[i] != asset_ids[i - 1]:
                start_indices[asset_ids[i]] = i

        return start_indices


    def reset(self, seed = seed, options=None, reset_hist=False):
        super().reset(seed= seed)
        self.logger.info("Environment reset")

        # Existing reset logic
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.fut_net_worth = self.initial_balance
        self.tokens_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Move to the next asset's starting index
        self.now_token += 1
        if self.now_token not in self.asset_start_indices:
            if self.mode == "train":
                self.now_token = list(self.asset_start_indices.keys())[0]  # Loop back to the first asset
            else:
                return None, None

        self.current_step = self.asset_start_indices[self.now_token] + self.window_length
        self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
        self.prev_token = self.now_token

        self.logger.info(f"Starting new episode with token {self.now_token} at step {self.current_step}")

        # Reset hist only if reset_hist is True
        if reset_hist:
            self.hist = {
                "current_step": [],
                'balance': [],
                'net_worth': [],
                'tokens_held': [],
                "token": [],
                "current_price": [],
                "reward": [],
                "action": [],
                'total_shares_sold': [],
                'total_sales_value': [],
            }

        observation = self._next_observation()
        info = {}
        return observation, info


    def _next_observation(self):
        frame = self.df.drop(columns = ['Close_orig']).loc[self.current_step - self.window_length + 1:self.current_step]
        # obs = np.concatenate([
        #     frame.values,
        #     [self.tokens_held],
        #     [self.balance],
        #     [self.net_worth]
        # ])
        obs = frame.values
        return obs.astype(np.float32)


    def step(self, action):
        self.reward = 0
        terminated = False
        truncated = False
        self.prev_token = self.now_token

        if isinstance(action, (list, np.ndarray)):
            action = action[0]

        self.logger.info(f"Step: {self.current_step}, Action taken: {action}")
        self._take_action(action)

        self.current_step += 1  # Move to the next time step
        
        if self.current_step >= self.total_steps:
            terminated = True
        else:
            self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
            if self.now_token != self.prev_token:
                self.logger.info(f"Token change at step {self.current_step}: {self.prev_token} -> {self.now_token}")
                self.now_token -=1
                terminated = True

        if not terminated:
            #  Теперь мы будем награждать модель используя данные за следующий шаг, а не предыдущий
            self.future_price = self.df.loc[self.current_step, 'Close_orig'] 
            self.fut_net_worth = self.balance + self.tokens_held * self.future_price

            if self.fut_net_worth != 0:
                net_worth_change = (self.fut_net_worth * 100 / self.net_worth) - 100
                price_change = (self.future_price * 100 / self.current_price) - 100
                initial_change = self.net_worth / self.initial_balance
                
                if price_change > 0:
                    self.reward += net_worth_change - (abs(abs(net_worth_change) - abs(price_change)) / 2)
                else:
                    self.reward += net_worth_change + (abs(abs(net_worth_change) - abs(price_change)) / 2)

                if self.net_worth > self.initial_balance:
                    self.reward += initial_change
                else:
                    self.reward += - (1 - initial_change)
                
                self.logger.info(f"step: {self.current_price - 1}, net_worth_change: {net_worth_change}, price_change: {price_change}, initial_change: {initial_change}, reward: {self.reward}")

                if self.net_worth < self.initial_balance * 0.5: # Только во время тренировки штрафуем за проеб половины баланса
                    if self.mode == "train":
                        self.logger.info("Net worth dropped below 50% of initial balance.")
                        terminated = True
            else:
                self.logger.info("fut_net_worth == 0")

        obs = self._next_observation()
        info = {}

        self.logger.info(f"Net worth: {self.net_worth}, Balance: {self.balance}, Reward: {self.reward}")

        self.hist["current_step"].append(self.current_step - 1)
        self.hist["balance"].append(self.balance)
        self.hist["net_worth"].append(self.net_worth)
        self.hist["tokens_held"].append(self.tokens_held)
        self.hist["token"].append(self.now_token)
        self.hist["current_price"].append(self.current_price)
        self.hist["reward"].append(self.reward)
        self.hist["action"].append(action)
        self.hist["total_shares_sold"].append(self.total_shares_sold)
        self.hist["total_sales_value"].append(self.total_sales_value)

        return obs, self.reward, terminated, truncated, info


    def _take_action(self, action):
        """
        Apply the continuous action to the current state.
        """
        self.current_price = self.df.loc[self.current_step, 'Close_orig']

        action = float(np.clip(action, -1, 1))

        if action < 0:
            proportion = -action  # Convert to positive
            shares_to_sell = int(self.tokens_held * proportion)
            self._sell(shares_to_sell)

        elif action > 0:
            proportion = action
            self._buy(proportion)
        else:
            pass  

        self.net_worth = self.balance + self.tokens_held * self.current_price


    def _buy(self, proportion):
        amount_to_spend = self.balance * proportion

        shares_to_buy = int(amount_to_spend / (self.current_price * (1 + self.fee_cost)))

        if shares_to_buy > 0:
            total_cost = shares_to_buy * self.current_price
            transaction_cost = total_cost * self.fee_cost
            total_cost += transaction_cost

            self.balance -= total_cost
            self.tokens_held += shares_to_buy

            self.logger.info(f"Bought {shares_to_buy} shares at price {self.current_price}")
            self.logger.info(f"Total cost: {total_cost}, Transaction cost: {transaction_cost}")
        else:
            self.logger.info("Not enough balance to buy.")


    def _sell(self, shares_to_sell):
        if shares_to_sell > self.tokens_held:
            shares_to_sell = self.tokens_held  # Can't sell more than held

        if shares_to_sell > 0:
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost

            self.balance += total_sale
            self.tokens_held -= shares_to_sell
            self.total_shares_sold += shares_to_sell
            self.total_sales_value += total_sale

            self.logger.info(f"Sold {shares_to_sell} shares at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.logger.info("No shares to sell.")


    def render(self, mode='human', close=False):
        profit = self.net_worth - self.initial_balance
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance:.2f}')
        print(f'Shares held: {self.tokens_held}')
        print(f'Net worth: {self.net_worth:.2f}')
        print(f'Profit: {profit:.2f}')

In [26]:
train_env = TradingEnv(train_df, mode = "train")

{0: 0, 1: 4387, 2: 8775, 3: 11706, 4: 16019, 5: 20386, 6: 24773, 7: 29152, 8: 32109, 9: 36418, 10: 40625, 11: 44207, 12: 47796, 13: 49643, 14: 53977, 15: 58319, 16: 62703, 17: 67087, 18: 70207, 19: 74595, 20: 78966}
-1


In [27]:
test_env = TradingEnv(test_df, mode = "test")

{21: 0, 22: 4093, 23: 8312, 24: 12700, 25: 16302}
20


In [28]:
# check_env(train_env)

# Visualisaton Functions

In [29]:
def plot_close_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id]
    
    # Построение графика Close к индексу DataFrame
    fig = px.line(asset_data, x=asset_data.index, y='Close_orig', 
                  title=f'Close Price for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Close': 'Close Price'})
    
    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "close_by_asset"))

In [30]:
def plot_reward_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['reward'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['reward'], mode='lines', name='Reward'))

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Reward = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Reward and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Reward')

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "reward_by_asset"))

In [31]:
def plot_price_change_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id].copy()
    
    # Вычисление процентного изменения цены (Close)
    asset_data['Price_Change_Percent'] = asset_data['Close_orig'].pct_change() * 100
    
    # Построение графика изменения цены в процентах
    fig = px.line(asset_data, x=asset_data.index, y='Price_Change_Percent', 
                  title=f'Price Change Percentage for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Price_Change_Percent': 'Price Change (%)'})
    
    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "price_change_by_asset"))

In [32]:
def plot_token_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['net_worth'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['net_worth'], mode='lines', name='Net Worth'))

    # Горизонтальная линия для net_worth = 1000
    fig.add_hline(y=1000, line_color="green", name='Net Worth = 1000')

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Net Worth = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Net Worth and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Net Worth')

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "token_data"))

In [33]:
def plot_action_counts(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token]
    
    # Подсчет количества каждого уникального действия для данного токена
    action_counts = token_data['action'].value_counts().reset_index()
    action_counts.columns = ['action', 'count']

    # Построение бар-чарта для отображения количества каждого действия
    fig = px.bar(action_counts, x='action', y='count', title=f'Count of Actions for {token}', labels={'action': 'Action', 'count': 'Count'})

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "action_counts"))

In [34]:
def plot_relative_change_by_token(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token].copy()

    # Вычисление относительного изменения для current_price и net_worth
    token_data['Price_Change_Percent'] = token_data['current_price'].pct_change() * 100
    token_data['NetWorth_Change_Percent'] = token_data['net_worth'].pct_change() * 100

    # Создание графика
    fig = go.Figure()

    # Линия для изменения current_price
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['Price_Change_Percent'],
                             mode='lines', name='Current Price Change (%)'))

    # Линия для изменения net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['NetWorth_Change_Percent'],
                             mode='lines', name='Net Worth Change (%)'))

    # Настройка заголовков и осей
    fig.update_layout(title=f'Relative Change of Current Price and Net Worth for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Change (%)')

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "relative_change_by_token"))

# Train Model

## Settings

In [ ]:
model = SAC(
    policy='MlpPolicy',  # Use a Multi-Layer Perceptron policy
    env=train_env,
    verbose=1,
    learning_rate=1e-4,  # Adjust learning rate if needed
    batch_size=512,      # Adjust batch size if needed
    tensorboard_log="./sac_tensorboard/",  # Directory for TensorBoard logs
    seed= seed,
)

In [30]:
# Set up the evaluation callback
from stable_baselines3.common.callbacks import EvalCallback
eval_callback = EvalCallback(
    test_env,                         # Evaluation environment
    best_model_save_path= Path["model_save"](model_num, data_num),   # Directory to save the best model
    log_path= Path["train_log"](model_num, data_num),               # Directory to save evaluation logs
    eval_freq=5000,                   # Evaluate every 5000 steps
    n_eval_episodes=3,                # Number of episodes to evaluate
    deterministic=True,               # Use deterministic actions during evaluation
    render=False                      # Disable rendering during evaluation
)

## Train

In [ ]:
model.learn(
    total_timesteps=nb_steps,
    log_interval=1000,        
    # callback=eval_callback   
    progress_bar= True,
)

In [32]:
model.save(Path["model_save"](model_num, data_num))

In [ ]:
train_hist = train_env.hist
train_hist_df = pd.DataFrame(train_hist)
print(len(train_hist["action"]))

In [ ]:
unique_values = train_hist_df['token'].unique()
unique_values

## Plots

In [ ]:
token = 20
plot_token_data(df = train_hist_df, token = token)
plot_close_by_asset(df= train_df, asset_id= token)
plot_price_change_by_asset(df= train_df, asset_id= token)
plot_relative_change_by_token(df = train_hist_df, token = token)

In [ ]:
train_hist_df.describe()

In [ ]:
train_hist_df

# Test Model

## Test

In [35]:
model = SAC.load(Path["model_save"](model_num, data_num))

/Users/mchomak/Projects/RL_for_Trading/.venv/lib/python3.11/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 9.61GB > 1.53GB
  warnings.warn(


In [36]:
obs, info = test_env.reset(reset_hist=True)  # Reset hist at the beginning
for _ in range(len(test_df)):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = test_env.step(action)

    if terminated or truncated:
        obs, info = test_env.reset(reset_hist=False)  # Do not reset hist
        if info is None:
            break

In [37]:
test_hist = test_env.hist
test_hist_df = pd.DataFrame(test_hist)
print(len(test_hist["action"]))

20433


In [38]:
test_hist_df

,current_step,balance,net_worth,tokens_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,48,22.835971,999.023812,42803992,21,0.000023,-2.448498,0.977164,0,0.0
1,49,0.742740,974.261034,43796936,21,0.000022,-0.007764,0.967476,0,0.0
2,50,0.083503,974.435563,43826559,21,0.000022,-0.700181,0.887595,0,0.0
3,51,0.011532,967.861507,43829815,21,0.000022,0.665250,0.861959,0,0.0
4,52,0.002273,974.611289,43830231,21,0.000022,-0.884353,0.803986,0,0.0
...,...,...,...,...,...,...,...,...,...,...
20428,20668,1000.000000,1000.000000,0,25,669.059508,-0.190832,-0.990377,0,0.0
20429,20669,1000.000000,1000.000000,0,25,671.613074,0.217984,-0.987453,0,0.0
20430,20670,1000.000000,1000.000000,0,25,668.685056,0.208857,-0.985990,0,0.0
20431,20671,1000.000000,1000.000000,0,25,665.891859,-0.039197,-0.979211,0,0.0


In [39]:
unique_values = test_hist_df['token'].unique()
unique_values

array([21, 22, 23, 24, 25])

## Plots

In [40]:
token = 25

In [41]:
plot_token_data(df = test_hist_df, token = token)

In [42]:
plot_close_by_asset(df= test_df, asset_id= token)

## Metrics

In [43]:
def evaluate_model(hist_df, test_df, initial_balance):
    """
    Evaluate the model's performance.

    Parameters:
    - hist_df: DataFrame containing the testing history.
    - test_df: DataFrame containing the test data.
    - initial_balance: Initial balance used in the environment.

    Returns:
    - report_df: DataFrame containing performance metrics per asset.
    - overall_metrics: Dictionary containing overall performance metrics.
    """

    # Ensure timestamps are in order
    hist_df = hist_df.sort_values('current_step').reset_index(drop=True)

    # List of assets
    assets = hist_df['token'].unique()

    # Initialize report DataFrame
    report = []

    for asset_id in assets:
        asset_hist = hist_df[hist_df['token'] == asset_id]
        asset_data = test_df[test_df['Asset_ID_encoded'] == asset_id]

        # Calculate total profit/loss
        final_net_worth = asset_hist['net_worth'].iloc[-1]
        total_profit = final_net_worth - initial_balance

        # Calculate ROI
        roi = (final_net_worth - initial_balance) / initial_balance * 100

        # Calculate Sharpe Ratio
        returns = asset_hist['net_worth'].pct_change().dropna()
        if returns.std() != 0:
            sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252)  # Assuming daily data
        else:
            sharpe_ratio = np.nan  # Undefined if no variance

        # Calculate Maximum Drawdown
        cumulative_returns = (1 + returns).cumprod()
        cumulative_max = cumulative_returns.cummax()
        drawdown = (cumulative_returns - cumulative_max) / cumulative_max
        max_drawdown = drawdown.min()

        # Calculate Win Rate
        trades = asset_hist[asset_hist['action'] != 0]
        wins = trades[trades['net_worth'].diff() > 0]
        win_rate = len(wins) / len(trades) * 100 if len(trades) > 0 else np.nan

        # Buy-and-Hold Strategy
        initial_price = asset_data['Close_orig'].iloc[0]
        final_price = asset_data['Close_orig'].iloc[-1]
        buy_and_hold_profit = (final_price - initial_price) * (initial_balance / initial_price)
        buy_and_hold_roi = (final_price - initial_price) / initial_price * 100

        # Ideal Strategy
        min_price = asset_data['Close_orig'].min()
        max_price = asset_data['Close_orig'].max()
        ideal_profit = (max_price - min_price) * (initial_balance / min_price)
        ideal_roi = (max_price - min_price) / min_price * 100

        # Collect metrics
        report.append({
            'Asset_ID': asset_id,
            'Total Profit': total_profit,
            'ROI (%)': roi,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown (%)': max_drawdown * 100,
            'Win Rate (%)': win_rate,
            'Buy-and-Hold Profit': buy_and_hold_profit,
            'Buy-and-Hold ROI (%)': buy_and_hold_roi,
            'Ideal Profit': ideal_profit,
            'Ideal ROI (%)': ideal_roi,
            'Asset Price Change (%)': (final_price - initial_price) / initial_price * 100,
        })

    # Create DataFrame from report
    report_df = pd.DataFrame(report)

    # Calculate overall averages for each column
    averages = report_df.mean(numeric_only=True)
    averages['Asset_ID'] = 'Average'  # Mark row as average

    # Append averages row to the DataFrame using pd.concat
    report_df = pd.concat([report_df, pd.DataFrame([averages])], ignore_index=True)

    # Calculate overall metrics
    overall_profit = report_df['Total Profit'].sum()
    overall_roi = (overall_profit / (initial_balance * len(assets))) * 100
    overall_sharpe = report_df['Sharpe Ratio'].mean()
    overall_win_rate = report_df['Win Rate (%)'].mean()
    overall_buy_and_hold_profit = report_df['Buy-and-Hold Profit'].sum()
    overall_buy_and_hold_roi = (overall_buy_and_hold_profit / (initial_balance * len(assets))) * 100

    overall_metrics = {
        'Total Profit': overall_profit,
        'ROI (%)': overall_roi,
        'Sharpe Ratio': overall_sharpe,
        'Win Rate (%)': overall_win_rate,
        'Buy-and-Hold Profit': overall_buy_and_hold_profit,
        'Buy-and-Hold ROI (%)': overall_buy_and_hold_roi,
    }

    return report_df, overall_metrics

- Asset_ID: Уникальный идентификатор актива (из столбца token), для которого рассчитываются метрики.

- Total Profit: Общий финансовый результат (прибыль или убыток) по данному активу. Рассчитывается как разница между конечной чистой стоимостью (net_worth) и начальным балансом (initial_balance).

- ROI (%): Доходность инвестиций (Return on Investment) в процентах. Показывает процентный прирост (или убыток) от начальной суммы баланса.

- Sharpe Ratio: Коэффициент Шарпа. Оценивает отношение доходности к риску (волатильности). Чем выше коэффициент Шарпа, тем лучше риск-корректированная доходность стратегии.

- Max Drawdown (%): Максимальная просадка в процентах. Это максимальное снижение стоимости актива от его исторического максимума. Отражает риски стратегии, связанные с падением стоимости.

- Win Rate (%): Процент прибыльных сделок. Это отношение количества прибыльных сделок к общему количеству сделок по активу, умноженное на 100.

- Buy-and-Hold Profit: Прибыль при стратегии "купить и держать". Показывает, сколько можно было бы заработать, если просто купить актив в начале и держать его до конца периода тестирования.

- Buy-and-Hold ROI (%): Доходность при стратегии "купить и держать". Процентный прирост от начальной цены актива, если его просто держать до конца периода.

- Ideal Profit: Идеальная прибыль. Это гипотетическая максимальная прибыль, которую можно было бы получить, если бы купили актив по минимальной цене и продали по максимальной цене за период.

- Ideal ROI (%): Идеальная доходность. Процентный прирост при идеальной стратегии, где покупка происходит по минимальной цене, а продажа — по максимальной.

- Asset Price Change (%): Изменение цены актива в процентах за период. Это процентное изменение цены от начальной до конечной за период тестирования.

In [44]:
report_df, overall_metrics = evaluate_model(test_hist_df, test_df, initial_balance = 1000)

/var/folders/vh/rjglqkc96s32q2n34g9g2l180000gn/T/ipykernel_63066/3864715686.py:85: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [45]:
report_df

,Asset_ID,Total Profit,ROI (%),Sharpe Ratio,Max Drawdown (%),Win Rate (%),Buy-and-Hold Profit,Buy-and-Hold ROI (%),Ideal Profit,Ideal ROI (%),Asset Price Change (%)
0,21,28.813180,2.881318,0.140526,-0.858966,47.960445,-279.352882,-27.935288,1596.459700,159.645970,-27.935288
1,22,1020.153760,102.015376,0.334681,-29.864954,1.702230,253802.131121,25380.213112,293446.958188,29344.695819,25380.213112
2,23,-0.573258,-0.057326,0.002586,-3.158201,31.497696,-152.651940,-15.265194,9635.856272,963.585627,-15.265194
3,24,16.837938,1.683794,0.131608,-1.628549,6.133934,594.470161,59.447016,2103.489180,210.348918,59.447016
4,25,0.000000,0.000000,NaN,0.000000,0.000000,233.832793,23.383279,2790.670479,279.067048,23.383279
5,Average,213.046324,21.304632,0.152350,-7.102134,17.458861,50839.685851,5083.968585,61914.686764,6191.468676,5083.968585


In [46]:
test_hist_df.describe()

,current_step,balance,net_worth,tokens_held,token,current_price,reward,action,total_shares_sold,total_sales_value
count,20433.000000,20433.000000,20433.000000,2.043300e+04,20433.000000,20433.000000,20433.000000,20433.000000,2.043300e+04,20433.000000
mean,10359.856702,998.010227,1009.621431,2.270519e+04,22.997015,78.111095,0.267938,-0.964059,8.654149e+06,389.887391
std,5965.411689,85.246593,52.732996,9.722407e+05,1.419961,154.761422,1.395839,0.171571,1.744665e+07,534.229405
min,48.000000,0.000277,954.064378,0.000000e+00,21.000000,0.000011,-25.116359,-1.000000,0.000000e+00,0.000000
25%,5204.000000,1000.000000,1000.000000,0.000000e+00,22.000000,0.008798,-0.294028,-0.998247,0.000000e+00,0.000000
50%,10360.000000,1000.000000,1000.000000,0.000000e+00,23.000000,0.272110,0.259047,-0.994575,0.000000e+00,0.000000
75%,15516.000000,1000.000000,1001.821283,1.000000e+00,24.000000,8.408479,0.986843,-0.982399,1.510000e+02,1028.812886
max,20672.000000,1028.813163,2334.019917,4.383032e+07,25.000000,674.820251,16.470793,0.999862,4.383032e+07,1269.456213


In [47]:
report_df.to_csv(Path["reports"](model_num, data_num, "test"), index= False)
test_hist_df.describe().to_csv(Path["reports"](model_num, data_num, "test_describe"), index= False)

In [48]:
overall_metrics

{'Total Profit': 1278.2779449724328,
 'ROI (%)': 25.565558899448654,
 'Sharpe Ratio': 0.15235029125421395,
 'Win Rate (%)': 17.45886082468679,
 'Buy-and-Hold Profit': 305038.1151049347,
 'Buy-and-Hold ROI (%)': 6100.762302098695}